In [0]:
%run ./utils_simulation

In [0]:

def summarize_sales_trend():
    df = spark.table("cpg_sales_gold_monthly")

    first = df.orderBy("year", "month").first()
    last = df.orderBy("year", "month").collect()[-1]

    growth = ((last.total_revenue - first.total_revenue) / first.total_revenue) * 100

    return (
        f"Sales have grown from {first.total_revenue:.2f} to "
        f"{last.total_revenue:.2f} over the period, "
        f"representing a growth of {growth:.1f}%. "
        f"This indicates an overall {'positive' if growth > 0 else 'negative'} trend."
    )

In [0]:
def summarize_promo_impact():
    df = spark.table("cpg_sales_gold_promo").collect()

    promo = [r for r in df if r.is_promo][0]
    non_promo = [r for r in df if not r.is_promo][0]

    uplift = ((promo.avg_units_sold - non_promo.avg_units_sold) / non_promo.avg_units_sold) * 100

    return (
        f"Promotions increase average units sold by approximately {uplift:.1f}%. "
        f"This shows that promotional campaigns are effective in driving demand."
    )

In [0]:
def summarize_price_hike():
    df = simulate_price_hike(10, 5)

    avg_revenue = df.selectExpr("avg(sim_revenue)").collect()[0][0]

    return (
        f"A simulated 10% price increase with a 5% demand drop results in "
        f"an average simulated revenue of {avg_revenue:.2f}. "
        f"This suggests that moderate price hikes may still improve revenue."
    )

In [0]:
def llm_classify_intent(question):
    q = question.lower()

    if any(k in q for k in ["price hike", "increase price", "price increase"]):
        return "PRICE_HIKE"

    if any(k in q for k in ["promo", "promotion"]):
        return "PROMO_ANALYSIS"

    if any(k in q for k in ["trend", "performance", "sales"]):
        return "TREND_ANALYSIS"

    return "UNKNOWN"

In [0]:
def agent_with_nlp_response(question):
    intent = llm_classify_intent(question)

    if intent == "TREND_ANALYSIS":
        return summarize_sales_trend()

    if intent == "PROMO_ANALYSIS":
        return summarize_promo_impact()

    if intent == "PRICE_HIKE":
        return summarize_price_hike()

    return (
        "I can help with sales trends, promotion impact, "
        "and price hike simulations. Please ask a related question."
    )

In [0]:
print("=" * 60)
print("SMART CPG DECISION SUPPORT AGENT (NLP UI)")
print("Ask business questions in natural language")
print("Type 'exit' to quit")
print("=" * 60)

while True:
    question = input("\nAsk a business question: ")

    if question.lower() == "exit":
        print("Exiting agent...")
        break

    response = agent_with_nlp_response(question)
    print("\nAgent Insight:")
    print(response)
    print("-" * 60)

SMART CPG DECISION SUPPORT AGENT (NLP UI)
Ask business questions in natural language
Type 'exit' to quit



Ask a business question:   How are sales performing?


Agent Insight:
Sales have grown from 29347.97 to 27329.39 over the period, representing a growth of -6.9%. This indicates an overall negative trend.
------------------------------------------------------------



Ask a business question:   Do promotions help sales?


Agent Insight:
Promotions increase average units sold by approximately 98.1%. This shows that promotional campaigns are effective in driving demand.
------------------------------------------------------------


...


Agent Insight:
A simulated 10% price increase with a 5% demand drop results in an average simulated revenue of 70.92. This suggests that moderate price hikes may still improve revenue.
------------------------------------------------------------



Ask a business question:  exit?


Agent Insight:
I can help with sales trends, promotion impact, and price hike simulations. Please ask a related question.
------------------------------------------------------------



Ask a business question:  exit

Exiting agent...
